[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). This notebook is released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).

# DIY Covid-19 Dashboard

In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [3]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
jsondata={"testcase1.json"}

In [4]:


def wrangle_data(data):
    with open("testcase1.json", "rt") as INFILE:
        data=json.load(INFILE)
    
        datalist=data['data']
        dates=[dictionary['date'] for dictionary in datalist ]
        dates.sort()

    def parse_date(datestring):
        return pd.to_datetime(datestring, format="%Y-%m-%d")

    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])

    index=pd.date_range(startdate, enddate, freq='D')
    testcasedf=pd.DataFrame(index=index, columns=['CumulativeCasesBySpecimenDate', 'CumulativePCRTestsBySpecimenDate', 'CumulativeLFDTestsBySpecimenDate'])



    for entry in datalist: 
        date=parse_date(entry['date'])
        for column in ['CumulativeCasesBySpecimenDate', 'CumulativePCRTestsBySpecimenDate','CumulativeLFDTestsBySpecimenDate']:
            if pd.isna(testcasedf.loc[date, column]): 
                value= float(entry[column]) if entry[column]!=None else 0.0
                testcasedf.loc[date, column]=value
   


    testcasedf.fillna(0.0, inplace=True)
    
    return testcasedf

# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in the cell as below:
testcasedf=wrangle_data(jsondata) # df is the dataframe for plotting

## Download current data

By pressing the button below, the data shown on the graph would refresh to up-to-date data from the [Public Health England](https://www.gov.uk/government/organisations/public-health-england) website.

In [5]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    filters = [
    'areaType=nation', 
    'areaName=England'
    ]


    structure = {
        "date": "date",
        "CumulativeCasesBySpecimenDate": "cumCasesBySpecimenDate",
        "CumulativePCRTestsBySpecimenDate": "cumPCRTestsBySpecimenDate",
        "CumulativeLFDTestsBySpecimenDate": "cumLFDTestsBySpecimenDate"
    }

    api = Cov19API(filters=filters, structure=structure)
    testcase=api.get_json()

    return {} # return data read from the API

In [6]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    apidata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global testcasedf
    testcasedf=wrangle_data(apidata)
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # you can omit this step in the first instance
    refresh_graph()
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. You can use icons
    # "unlink" or "times" and change the button text to "Unavailable" in case the 
    # api call fails.
    apibutton.icon="check"
    # apibutton.disabled=True

    
apibutton=wdg.Button(
    description='Refresh data', # you may want to change this...
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Click to download current Public Health England data",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='download'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

display(apibutton)

# run all cells before clicking on this button

Button(button_style='info', description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click t…

## Graphs and Analysis

The graph shown below pictures the comparsion of the cumulative amount of LFD and PCR tests by specimen date conducted versus the cumulative number of cases by specimen date of COVID-19 in England since the beginning of the pandemic.
The drop-down menu allows the user to see one type of data. In case of the need to see all types or chosen types at the same time, the options should be clicked while the user is holding the CTRL button.

In [7]:


def testcase_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        testcasedf[list(gcols)].plot(logy=logscale, xlabel="Date", ylabel="The number of cases and tests")
        plt.show()
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
    
series=wdg.SelectMultiple(
    options=['CumulativeCasesBySpecimenDate', 'CumulativeLFDTestsBySpecimenDate', 'CumulativePCRTestsBySpecimenDate'],
    value=['CumulativeCasesBySpecimenDate', 'CumulativeLFDTestsBySpecimenDate', 'CumulativePCRTestsBySpecimenDate'],
    rows=3,
    description='Statistics:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False    
)


controls=wdg.HBox([series, scale])


def refresh_graph():
    current=series.value
    if current==series.options[0]:
        other=series.options[1]
    else:
        other=series.options[0]
    series.value=other # forces the redraw
    series.value=current # now we can change it back


    
graph=wdg.interactive_output(testcase_graph, {'gcols': series, 'gscale': scale})

display(controls, graph)


Output()

**Author and Copyright Notice** Wiktoria Powyszynska, Queen Mary University of London. Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england) and on the [DIY Covid Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash), Copyright (C) Fabrizio Smeraldi 2020,2023. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).